In [1]:
! pip install -q transformers datasets scikit-learn pandas

In [2]:
import pandas as pd
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import TrainingArguments

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\Users\ngzhi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [15]:
file_path = r"FinancialPhraseBank-v1.0\FinancialPhraseBank-v1.0\Sentences_66Agree.txt"

# Use cp1252 encoding instead of utf-8
with open(file_path, 'r', encoding='cp1252') as f:
    lines = f.readlines()

# Inspect
print(lines[:5])

sentences = []
labels = []

for line in lines:
    line = line.strip()
    if not line:
        continue
    if "@positive" in line:
        label = "positive"
        text = line.replace("@positive", "").strip()
    elif "@negative" in line:
        label = "negative"
        text = line.replace("@negative", "").strip()
    elif "@neutral" in line:
        label = "neutral"
        text = line.replace("@neutral", "").strip()
    else:
        continue  # skip malformed lines

    sentences.append(text)
    labels.append(label)

print(f"Loaded {len(sentences)} sentences.")

# 3️⃣ Map labels to integers
df = pd.DataFrame({"text": sentences, "label": labels})
df['label'] = df['label'].map({"negative": 0, "neutral": 1, "positive": 2})

# Inspect
print(df.head())
print(df['label'].value_counts())


# Split into train/test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])


# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")

def preprocess(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128)

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)



['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .@neutral\n', 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .@neutral\n', 'With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .@positive\n', "According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .@positive\n", "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .@positive\n"]
Loaded 

NameError: name 'Dataset' is not defined

In [ ]:
# Load FinBERT pre-trained for sentiment classification
model = AutoModelForSequenceClassification.from_pretrained(
    "yiyanghkust/finbert-tone",
    num_labels=3  # positive, neutral, negative
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

In [5]:
training_args = TrainingArguments(
    output_dir="./finbert_full_ft",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


NameError: name 'TrainingArguments' is not defined